# Multilayer Perceptron (MLP) for the names dataset 

In [1]:
import torch
import torch.functional as F
import matplotlib.pyplot as plt



In [4]:
words = open('data/names.txt', 'r').read().split()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [5]:
len(words)

32033

## Tokenization

In [10]:
chars = sorted(list(set(''.join(words))))
stringToIndex = {char:index + 1 for index, char in enumerate(chars)}
stringToIndex['.'] = 0
indexToString = {index:char for char, index in stringToIndex.items()}
print(indexToString)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [52]:
# Building the dataset
block_size = 3
X, Y = [], []
for word in words:
    # print(word)
    context = [0] * block_size
    for ch in word + '.':
        ix = stringToIndex[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(indexToString[i] for i in context), '------->', indexToString[ix])
        context = context[1:] + [ix] # Crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([228146, 3])

In [55]:
# Embedding layer
C = torch.randn((27, 2))
emb = C[X]

# Weights to the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn((100,))

## 3 ways to reshape the embedding layer

In [68]:
# Not useful, hard coded. What if my block_size changes?
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1)

tensor([[-0.9799, -0.1426, -0.9799, -0.1426, -0.9799, -0.1426],
        [-0.9799, -0.1426, -0.9799, -0.1426,  1.3232, -1.5216],
        [-0.9799, -0.1426,  1.3232, -1.5216, -0.4166,  0.1294],
        ...,
        [-0.2963, -0.5040, -0.2963, -0.5040,  0.5572,  0.9360],
        [-0.2963, -0.5040,  0.5572,  0.9360, -0.2963, -0.5040],
        [ 0.5572,  0.9360, -0.2963, -0.5040,  0.0260, -0.4588]])

In [69]:
# Not efficient, we are creating another tensor
torch.cat(torch.unbind(emb, dim = 1), dim = 1)

tensor([[-0.9799, -0.1426, -0.9799, -0.1426, -0.9799, -0.1426],
        [-0.9799, -0.1426, -0.9799, -0.1426,  1.3232, -1.5216],
        [-0.9799, -0.1426,  1.3232, -1.5216, -0.4166,  0.1294],
        ...,
        [-0.2963, -0.5040, -0.2963, -0.5040,  0.5572,  0.9360],
        [-0.2963, -0.5040,  0.5572,  0.9360, -0.2963, -0.5040],
        [ 0.5572,  0.9360, -0.2963, -0.5040,  0.0260, -0.4588]])

In [67]:
# Efficient way
emb.view(-1, block_size * C.shape[1])

tensor([[-0.9799, -0.1426, -0.9799, -0.1426, -0.9799, -0.1426],
        [-0.9799, -0.1426, -0.9799, -0.1426,  1.3232, -1.5216],
        [-0.9799, -0.1426,  1.3232, -1.5216, -0.4166,  0.1294],
        ...,
        [-0.2963, -0.5040, -0.2963, -0.5040,  0.5572,  0.9360],
        [-0.2963, -0.5040,  0.5572,  0.9360, -0.2963, -0.5040],
        [ 0.5572,  0.9360, -0.2963, -0.5040,  0.0260, -0.4588]])

In [72]:
h = torch.tanh(emb.view(-1, block_size * C.shape[1])@(W1) + b1)
h.shape

torch.Size([228146, 100])